In [1]:
import pandas as pd, numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from stock_keys import api_key
from stock_saver import Stock
import stock_chart as sc
import stock_indicators
import math

ticker = 'NSE:INFY'
intraday = True
day, month, year = 28, 6, 2019
st = Stock(ticker, api_key, intraday, None)
df = pd.DataFrame(data=st.df)
df = df[df.index.map(lambda x: (df.date[x].year) == year and (df.date[x].month) == month and (df.date[x].day) == day)]
print('Filtered values:', len(df.index))
output_notebook()

[NSE:INFY] Intraday : 2019-06-28 15:29:00
[NSE:INFY] 641 values added.
Filtered values: 375


Loading BokehJS ...

In [2]:
# power 1 seems to be working fine as it identifies more important points.
for power in [1]:
    stock_indicators.compute_momentums_and_derivatives(df, power, 0.96)
    p = figure(x_axis_type="datetime", plot_width=1000, plot_height=350, x_range = (df.date.min(), df.date.max()))
    stock_indicators.compute_correlation(df)
    p.line('date', 'cor', source = ColumnDataSource(df), line_color="pink")
    # keep t = 40 here.
    stock_indicators.compute_rolling_correlation(df, t=40)
    p.line('date', 'b_ema', source = ColumnDataSource(df), line_color="green")
    p.line('date', 's_ema', source = ColumnDataSource(df), line_color="red")
    p.line('date', 'closeness_sma', source = ColumnDataSource(df), line_color="gray")
    p.line('date', '0', source = ColumnDataSource(df), line_color="black")
#     p.line('date', 'river_up', source = ColumnDataSource(df), line_color="black")
    p.line('date', 'rolling_correlation', source = ColumnDataSource(df), line_color="orange")
    blues = stock_indicators.compute_blues(df)
    p.circle(blues['date'], blues['value'], size=8, color="blue", alpha=0.5)
    stock_indicators.compute_rolling_correlation(df, t=15)
#     p.line('date', 'rolling_correlation', source = ColumnDataSource(df), line_color="blue", alpha=0.5)
    p = sc.set_title(p, st.ticker + ' ' + str(power))
    show(p)
    
    p = figure(x_axis_type="datetime", plot_width=1000, plot_height=350, x_range = (df.date.min(), df.date.max()), y_range = (df.low.min(), df.high.max()))
    p = sc.set_title(p, st.ticker)
    # trends lies between local maximas and minimas of the 10min correlation
    p = sc.draw_price(p, df)
    p = sc.draw_volume(p, df, False)
    p.circle(blues['date'], blues['price'], size=15, color="orange", alpha=0.8)
    show(p)
    # print the correlations at the end.
    # orange points seem to either support already existed trend everywhere or oppose it, after 11am with t = 40.
    # seems to be supporting it, when closeness is very tight right from the start. ex:- MSFT 30-5.
    # and seems to indicate reversal when the closeness is not so tight. ex:- GOOGL 30-5. Exit points need to be profit/risk based in this case.
    # or the trades could be done point to point but will require reading candles at every point.

/home/demq/.anaconda3/envs/dq/lib/python3.7/site-packages/scipy/stats/stats.py:3399: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
